In [ ]:
spark.sql("""DROP TABLE IF EXISTS OLTP_customers""")
spark.sql("""DROP TABLE IF EXISTS SCD2_customers""")

spark.sql(""" 
    CREATE TABLE OLTP_customers (
    customer_id int PRIMARY KEY, 
    customer_name VARCHAR(30), 
    customer_adress VARCHAR(30),
    update_time TIMESTAMP)
""")

spark.sql(""" 
    CREATE TABLE SCD2_customers (
    customer_id INT, 
    customer_name VARCHAR(30), 
    customer_adress VARCHAR(30),
    start_time TIMESTAMP,
    end_time TIMESTAMP,
    current_flag CHAR(1));
""")

spark.sql(""" 
    INSERT INTO OLTP_customers
    VALUES 
    (1,'Jenny','5th Avenue', CURRENT_TIMESTAMP()),
    (2,'Joseph','High Way', CURRENT_TIMESTAMP()),
    (3,'Marcus','Higher Hills',CURRENT_TIMESTAMP())
 """)
 
# Creating SCD type_2 Function
def SCD_type_2():
    spark.sql("""
        MERGE into SCD2_customers AS SCD
        USING OLTP_customers AS OLTP
        ON SCD.customer_id = OLTP.customer_id
        -- Update last exisitng row
            WHEN MATCHED AND (SCD.customer_name <> OLTP.customer_name OR SCD.customer_adress <> OLTP.customer_adress )
                         AND SCD.current_flag = 'Y' THEN 
                UPDATE SET
                    SCD.end_time = OLTP.update_time - INTERVAL 1 second,
                    SCD.current_flag = 'N'
        -- Insert Scenario
            WHEN NOT MATCHED THEN
                INSERT(customer_id,customer_name,customer_adress,start_time,end_time,current_flag)
                VALUES(OLTP.customer_id,OLTP.customer_name,OLTP.customer_adress,OLTP.update_time,NULL,'Y')
         -- Delete Scenario
            WHEN NOT MATCHED BY SOURCE AND SCD.end_time IS NULL THEN
                UPDATE SET SCD.end_time = current_timestamp()
        """)
    # Inserting new updated records
    spark.sql("""
        MERGE into SCD2_customers AS SCD
        USING OLTP_customers AS OLTP
        ON SCD.customer_id = OLTP.customer_id
            AND SCD.customer_name = OLTP.customer_name
            AND SCD.customer_adress = OLTP.customer_adress
         WHEN NOT MATCHED THEN
                INSERT(customer_id,customer_name,customer_adress,start_time,end_time,current_flag)
                VALUES(OLTP.customer_id,OLTP.customer_name,OLTP.customer_adress,OLTP.update_time,NULL,'Y')
            """)

display(spark.sql("SELECT * FROM OLTP_customers"))
display(spark.sql("SELECT * FROM SCD2_customers"))

# Inserting new Customer
spark.sql(""" INSERT INTO OLTP_customers VALUES (4,'Jerry','Hawai', CURRENT_TIMESTAMP()) """)
SCD_type_2()
display(spark.sql("SELECT * FROM OLTP_customers"))
display(spark.sql("SELECT * FROM SCD2_customers"))

# Updating existing Customer
spark.sql(""" UPDATE oltp_customers SET customer_adress = 'Forest' WHERE customer_id = 2 """)
SCD_type_2()
display(spark.sql("SELECT * FROM OLTP_customers"))
display(spark.sql("SELECT * FROM SCD2_customers"))

# Deleting Customer
spark.sql(""" DELETE FROM oltp_customers WHERE customer_id = 1 """)
SCD_type_2()
display(spark.sql("SELECT * FROM OLTP_customers"))
display(spark.sql("SELECT * FROM SCD2_customers"))